In [32]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.preprocessing import PowerTransformer


In [33]:
data=pd.read_csv("/home/aaditya/Downloads/new-main/train_dK8Auve.csv")

In [34]:
data.Office_PIN.nunique()

98

In [35]:
data=data.drop('ID',axis=1)

In [36]:
#data

In [37]:
data.Applicant_Occupation.fillna("Others",inplace=True)
data.Applicant_Marital_Status.fillna("Applicant_Marital_StatusMissing",inplace=True)
data.Applicant_Gender.fillna("Applicant_GenderMissing",inplace=True)
data.Applicant_Qualification.fillna("Others",inplace=True)
data.Manager_Gender.fillna("Manager_GenderMissing",inplace=True)
data.Manager_Status.fillna("Manager_StatusMissing",inplace=True)

In [38]:
data['Application_Receipt_Date'] = pd.to_datetime(data['Application_Receipt_Date'], format =  '%m/%d/%Y')
data['Applicant_BirthDate'] = pd.to_datetime(data['Applicant_BirthDate'], format =  '%m/%d/%Y')
data['Manager_DoB'] = pd.to_datetime(data['Manager_DoB'], format =  '%m/%d/%Y')
data['Manager_DOJ'] = pd.to_datetime(data['Manager_DOJ'], format =  '%m/%d/%Y')

In [39]:
data['Application_Receipt_Date_year'] = data['Application_Receipt_Date'].dt.year
data['Application_Receipt_Date_month'] = data['Application_Receipt_Date'].dt.month
data['Application_Receipt_Date_day'] = data['Application_Receipt_Date'].dt.day
data['Application_Receipt_Date_hour'] = data['Application_Receipt_Date'].dt.hour
data['Application_Receipt_Date_minute'] = data['Application_Receipt_Date'].dt.minute

data['Applicant_BirthDate_day'] = data['Applicant_BirthDate'].dt.day
data['Applicant_BirthDate_year'] = data['Applicant_BirthDate'].dt.year
data['Applicant_BirthDate_month'] = data['Applicant_BirthDate'].dt.month
data['Applicant_BirthDate_hour'] = data['Applicant_BirthDate'].dt.hour
data['Applicant_BirthDate_minute'] = data['Applicant_BirthDate'].dt.minute



In [40]:
data['Manager_Joining_Designation']=data['Manager_Joining_Designation'].map(lambda x:float(str(x)[-1]) if (str(x)[-1]!='n')& (str(x)[-1]!='r') else float('nan'))
data['Manager_Current_Designation']=data['Manager_Current_Designation'].map(lambda x:float(str(x)[-1]) if (str(x)[-1]!='n')& (str(x)[-1]!='r') else float('nan'))


In [41]:
data['Manager_Promotion']=data['Manager_Current_Designation']-data['Manager_Joining_Designation']

In [42]:
data['PinDifference'] =abs(data['Office_PIN']-data['Applicant_City_PIN'])

In [43]:
data["Business_Sourced"].value_counts()

0    6260
1    3267
Name: Business_Sourced, dtype: int64

In [44]:
data[data["Business_Sourced"]==0][:3267]

In [45]:
data[data["Business_Sourced"]==1][:3267]

In [46]:
data_new =pd.DataFrame()
data_new = pd.concat([data[data["Business_Sourced"]==0][:3267],data[data["Business_Sourced"]==1][:3267]])

In [47]:
data_new.shape

(6534, 34)

In [48]:
data_new["Business_Sourced"].value_counts()

1    3267
0    3267
Name: Business_Sourced, dtype: int64

In [49]:
del data

In [50]:
data = pd.DataFrame()
data = data_new

In [51]:
data["Business_Sourced"].value_counts()

1    3267
0    3267
Name: Business_Sourced, dtype: int64

In [52]:
import datetime as dt

#data['Manager_age'] = (np.floor((pd.to_datetime(data['Manager_DoB']) - pd.to_datetime(data['Manager_DOJ'])).dt.days / 365.25)).astype(int)
data['Manager_age']=abs(data['Application_Receipt_Date'].dt.year - data['Manager_DoB'].dt.year)
data['Applicant_age']=abs(data['Application_Receipt_Date'].dt.year - data['Applicant_BirthDate'].dt.year)


In [53]:
for column in data:
    if data[column].dtype == "float64" or data[column].dtype == "int64":
        data[column].fillna(data[column].median(), inplace=True)

In [54]:
data=data.drop(columns=['Application_Receipt_Date','Applicant_BirthDate','Applicant_Qualification','Manager_DoB','Manager_DOJ'])

In [55]:
data.shape

(6534, 31)

In [56]:
data['Manager_age'].dtype

dtype('float64')

In [57]:
target=data['Business_Sourced']

In [58]:
num=data[['Manager_Business','Manager_Num_Products','Manager_Business2','Manager_Num_Products2','Application_Receipt_Date_year','Application_Receipt_Date_month','Application_Receipt_Date_day','Applicant_BirthDate_year','Applicant_BirthDate_month','Applicant_BirthDate_day','Manager_age','Applicant_age']]

In [59]:
label=data[['Office_PIN','Applicant_City_PIN','Manager_Grade','Manager_Num_Coded','Manager_Num_Application','Manager_Current_Designation','PinDifference','Manager_Promotion']]

In [60]:
from sklearn import preprocessing
label_encode=label.apply(LabelEncoder().fit_transform)

In [61]:
label_encode.shape

(6534, 8)

In [62]:
one_hot=data[['Applicant_Gender','Applicant_Marital_Status','Applicant_Occupation','Manager_Joining_Designation','Manager_Status','Manager_Gender']]

In [63]:
enc = pd.get_dummies(one_hot,drop_first=True)

In [64]:
#enc

In [65]:
enc.shape

(6534, 15)

In [66]:
data_s=pd.concat([label_encode,enc,num],axis=1)

In [67]:
#data_s.columns

In [68]:
#data_s

In [69]:
scaler= StandardScaler()
x=scaler.fit_transform(data_s)
x

/home/aaditya/.local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/aaditya/.local/lib/python2.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([[ 1.54336258,  1.66810951, -0.22286315, ...,  0.64507065,
        -0.94731626,  0.35403175],
       [ 1.15690494,  1.65591219, -1.12341221, ...,  0.293018  ,
         0.50137256,  0.83427662],
       [ 1.33256751,  1.43771555,  0.67768591, ..., -1.23254346,
        -0.94731626, -1.27880079],
       ...,
       [ 0.10292955, -0.10456876, -0.22286315, ..., -0.41108729,
         0.34040714, -0.22226209],
       [ 1.0515074 ,  1.12058503,  0.67768591, ..., -0.05903464,
         0.66233799, -0.41436003],
       [ 1.15690494,  1.66946477, -0.22286315, ..., -0.29373641,
         0.34040714,  0.73822764]])

In [70]:
x.shape

(6534, 35)

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, target, test_size=0.30, random_state=0)

In [91]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.kernel_approximation import Nystroem
###############################################3
#clf = svm.SVC(kernel='linear', class_weight='balanced')
clf =RandomForestClassifier(n_estimators=1000,random_state=0)#DecisionTreeClassifier(random_state=0)
#clf=KNeighborsClassifier(n_neighbors=2)
#clf = svm.SVC()
clf.fit(X_train , y_train)
# sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
# sig_clf.fit(train_x_onehotCoding, train_y)
y_pred = clf.predict(X_test)

In [92]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt 
logit_roc_auc = roc_auc_score(y_test, y_pred)
print(logit_roc_auc)

0.7602675320892884


In [90]:
#Random forest
from sklearn.ensemble import RandomForestClassifier as rf
rf_model = rf()#ccp_alpha=0, criterion='entropy', max_depth=10,
                #       max_leaf_nodes=97, random_state=10,class_weight='balanced')

 

# fitting the model
rf_model.fit(X_train,y_train)

 

# make training prediction
train_yhat = rf_model.predict(X_train)
train_score = roc_auc_score(y_train, train_yhat)

 

# make test prediction
test_yhat = rf_model.predict(X_test)
test_score = roc_auc_score(y_test, test_yhat)

 

train_score, test_score
print(roc_auc_score(y_test,test_yhat))

0.7424197507749276


/home/aaditya/.local/lib/python2.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [330]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier

#reg = DecisionTreeRegressor(random_state=0)

from sklearn.ensemble import GradientBoostingClassifier
grad=GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)
#clf = RandomForestClassifier(n_estimators=500)
grad.fit(X_train,y_train)
pred2 = grad.predict(X_test)

In [331]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
dt_roc_auc = roc_auc_score(y_test, pred2)
print(dt_roc_auc)

0.5223324002821563


In [332]:
from xgboost import XGBClassifier
model = XGBClassifier(max_depth= 8, n_estimators = 1000, eta = 0.01, eval_metric= "auc" )
model.fit(X_train,y_train)
y_predict = model.predict_proba(X_test)

ImportError: No module named xgboost

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
dt_roc_auc = roc_auc_score(y_test, y_predict)
print(dt_roc_auc)

In [ ]:
model.fit(x_train_data,y_train_data)
y_predict = model.predict_proba(test)

In [190]:
testdata=pd.read_csv("/home/aaditya/Downloads/new-main/test_UyrvqPb.csv")

In [191]:
testdata.shape

(5045, 22)

In [192]:
testdata.Applicant_Occupation.fillna("Others",inplace=True)
testdata.Applicant_Marital_Status.fillna("Applicant_Marital_StatusMissing",inplace=True)
testdata.Applicant_Gender.fillna("Applicant_GenderMissing",inplace=True)
testdata.Applicant_Qualification.fillna("Others",inplace=True)
testdata.Manager_Gender.fillna("Manager_GenderMissing",inplace=True)
testdata.Manager_Status.fillna("Manager_StatusMissing",inplace=True)


In [193]:
testdata['Application_Receipt_Date'] = pd.to_datetime(testdata['Application_Receipt_Date'], format =  '%m/%d/%Y')
testdata['Applicant_BirthDate'] = pd.to_datetime(testdata['Applicant_BirthDate'], format =  '%m/%d/%Y')
testdata['Manager_DoB'] = pd.to_datetime(testdata['Manager_DoB'], format =  '%m/%d/%Y')
testdata['Manager_DOJ'] = pd.to_datetime(testdata['Manager_DOJ'], format =  '%m/%d/%Y')

In [198]:
testdata['Application_Receipt_Date_year'] = testdata['Application_Receipt_Date'].dt.year
testdata['Application_Receipt_Date_month'] = testdata['Application_Receipt_Date'].dt.month
testdata['Application_Receipt_Date_day'] = testdata['Application_Receipt_Date'].dt.day
testdata['Application_Receipt_Date_hour'] = testdata['Application_Receipt_Date'].dt.hour
testdata['Application_Receipt_Date_minute'] = testdata['Application_Receipt_Date'].dt.minute

testdata['Applicant_BirthDate_year'] = testdata['Applicant_BirthDate'].dt.year
testdata['Applicant_BirthDate_month'] = testdata['Applicant_BirthDate'].dt.month
testdata['Applicant_BirthDate_day'] = testdata['Applicant_BirthDate'].dt.day
testdata['Applicant_BirthDate_hour'] = testdata['Applicant_BirthDate'].dt.hour
testdata['Applicant_BirthDate_minute'] = testdata['Applicant_BirthDate'].dt.minute




In [199]:
testdata['Manager_Joining_Designation']=testdata['Manager_Joining_Designation'].map(lambda x:float(str(x)[-1]) if (str(x)[-1]!='n')& (str(x)[-1]!='r') else float('nan'))
testdata['Manager_Current_Designation']=testdata['Manager_Current_Designation'].map(lambda x:float(str(x)[-1]) if (str(x)[-1]!='n')& (str(x)[-1]!='r') else float('nan'))


In [200]:
testdata['Manager_Promotion']=testdata['Manager_Current_Designation']-testdata['Manager_Joining_Designation']

In [201]:
testdata['PinDifference'] =abs(testdata['Office_PIN']-testdata['Applicant_City_PIN'])

In [202]:
import datetime as dt

#data['Manager_age'] = (np.floor((pd.to_datetime(data['Manager_DoB']) - pd.to_datetime(data['Manager_DOJ'])).dt.days / 365.25)).astype(int)
testdata['Manager_age']=abs(testdata['Application_Receipt_Date'].dt.year - testdata['Manager_DoB'].dt.year)
testdata['Applicant_age']=abs(testdata['Application_Receipt_Date'].dt.year - testdata['Applicant_BirthDate'].dt.year)


In [203]:
for column in testdata:
    if testdata[column].dtype == "float64" or testdata[column].dtype == "int64":
        testdata[column].fillna(testdata[column].median(), inplace=True)


In [204]:
testdata=testdata.drop(columns=['Application_Receipt_Date','Applicant_BirthDate','Manager_DoB','Manager_DOJ','Applicant_Qualification'])

In [205]:
test_num=testdata[['Manager_Business','Manager_Num_Products','Manager_Business2','Manager_Num_Products2','Application_Receipt_Date_year','Application_Receipt_Date_month','Application_Receipt_Date_day','Applicant_BirthDate_year','Applicant_BirthDate_month','Applicant_BirthDate_day','Manager_age','Applicant_age']]

In [206]:
test_label=testdata[['Office_PIN','Applicant_City_PIN','Manager_Grade','Manager_Num_Coded','Manager_Num_Application','Manager_Current_Designation','PinDifference','Manager_Promotion']]

In [207]:
from sklearn import preprocessing
test_label_encode=test_label.apply(LabelEncoder().fit_transform)

In [208]:
test_label_encode.shape

(5045, 8)

In [209]:
test_one_hot=testdata[['Applicant_Gender','Applicant_Marital_Status','Applicant_Occupation','Manager_Joining_Designation','Manager_Status','Manager_Gender']]

In [210]:
test_enc = pd.get_dummies(test_one_hot,drop_first=True)

In [211]:
enc.columns

Index([u'Manager_Joining_Designation', u'Applicant_Gender_F',
       u'Applicant_Gender_M', u'Applicant_Marital_Status_D',
       u'Applicant_Marital_Status_M', u'Applicant_Marital_Status_S',
       u'Applicant_Marital_Status_W', u'Applicant_Occupation_Others',
       u'Applicant_Occupation_Salaried', u'Applicant_Occupation_Self Employed',
       u'Applicant_Occupation_Student',
       u'Manager_Status_Manager_StatusMissing', u'Manager_Status_Probation',
       u'Manager_Gender_M', u'Manager_Gender_Manager_GenderMissing'],
      dtype='object')

In [212]:
data_t=pd.concat([test_label_encode,test_enc,test_num],axis=1)

In [213]:
data_t.columns


Index([u'Office_PIN', u'Applicant_City_PIN', u'Manager_Grade',
       u'Manager_Num_Coded', u'Manager_Num_Application',
       u'Manager_Current_Designation', u'PinDifference', u'Manager_Promotion',
       u'Manager_Joining_Designation', u'Applicant_Gender_F',
       u'Applicant_Gender_M', u'Applicant_Marital_Status_D',
       u'Applicant_Marital_Status_M', u'Applicant_Marital_Status_S',
       u'Applicant_Marital_Status_W', u'Applicant_Occupation_Others',
       u'Applicant_Occupation_Salaried', u'Applicant_Occupation_Self Employed',
       u'Applicant_Occupation_Student',
       u'Manager_Status_Manager_StatusMissing', u'Manager_Status_Probation',
       u'Manager_Gender_M', u'Manager_Gender_Manager_GenderMissing',
       u'Manager_Business', u'Manager_Num_Products', u'Manager_Business2',
       u'Manager_Num_Products2', u'Application_Receipt_Date_year',
       u'Application_Receipt_Date_month', u'Application_Receipt_Date_day',
       u'Applicant_BirthDate_year', u'Applicant_BirthDat

In [214]:
scaler= StandardScaler()
xx=scaler.fit_transform(data_t)
xx

array([[-0.47100331, -0.53159598, -1.40950835, ..., -0.99349413,
        -0.73579325, -0.73064253],
       [ 1.31587199,  1.48410252, -1.40950835, ..., -0.28966836,
         0.56466912,  0.61830357],
       [ 1.31587199, -0.08384656, -0.32120568, ...,  0.29685311,
         0.0073281 , -0.92334911],
       ...,
       [ 0.98343007,  0.97522126, -0.32120568, ...,  0.76607028,
        -0.55001291, -0.15252277],
       [-1.09433191, -1.07186927, -0.32120568, ...,  1.00067887,
        -0.55001291, -0.92334911],
       [-1.46832906, -1.38578953, -0.32120568, ..., -0.17236407,
        -1.47891461, -0.92334911]])

In [215]:
xx.shape

(5045, 35)

In [221]:
y_pred = clf.predict(xx)

In [217]:
y_pred

array([1, 1, 0, ..., 1, 1, 0])

In [219]:
final_data=pd.DataFrame()
final_data['ID']=testdata['ID']
final_data['Business_Sourced']=y_pred
final_data

,ID,Business_Sourced
0,FIN1009528,1
1,FIN1009529,1
2,FIN1009530,0
3,FIN1009531,0
4,FIN1009532,0
5,FIN1009533,1
6,FIN1009534,0
7,FIN1009535,1
8,FIN1009536,0
9,FIN1009537,1


In [222]:
logit_roc_auc = roc_auc_score(y_test, y_pred)
print(logit_roc_auc)

ValueError: Found input variables with inconsistent numbers of samples: [3335, 5045]